## America's Working Class and How It Helped Swing the 2016 Election

Many pundits have argued that this past election was decided by America's working class. It has always been the case that jobs and the economy play a pivotal role in influencing the voting decisions of many Americans. With the Great Recession of 2008-2011, and an increasing shift towards a globalized economy, traditionally blue-collar industries such as construction, manufacturing, and transportation and trade felt the brunt of those effects through the loss of jobs. 

We hypothesize in this project that the people most affected by these changes - America's working class - responded to these changes with their votes, and may correlate with whether a state went red or blue in 2016. While obviously it's hard to definitively pin down a cause just based on how people voted, we want to compare labor department data about umeployment in certain sectors and voting data from certain states to see if we can find a trend or come to any interesting conclusions.

For our project, we look towards a number of sources to aid in our analysis. First, on the labor statistics front, we looked towards the [Bureau of Labor Statistics](http://www.bls.gov/bls/proghome.htm) and, specifically, the [Bureau of Labor Statistics Public Data API](http://www.bls.gov/developers/api_signature_v2.htm) to collect data on unemployment over the industries most impacted by the recession, and break down this data by state. We will combine this feature with the recovery rate of a given state by 2016. Second, for election data, we turn towards major news sources like [Politico](http://www.politico.com/) and the [New York Times](http://www.nytimes.com) to collect voter numbers from each state during the 2008, 2012, and 2016 elections. 

## Analyzing the Election

Though both Politico and the New York Times, as many news sources do, ultimately draw their data from the Associated Press, API keys for AP aren't free, so we'll have to take a slightly more indirect approach by scraping the news pages themselves for the data instead of getting it directly from AP. Unfortunately, due to formatting changes over the years, it's a little more difficult than we'd like it to be to fetch voter data from the 2008, 2012, and 2016 elections with one script. However, it's not too hard to write scripts for each of those elections individually.

In [1]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
# pd.options.display.max_seq_items = 2000
# pd.set_option('display.height', 1000)
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

In [138]:
import pandas as pd
import numpy as np
import requests
from sklearn.svm import SVC
import time
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

years = [#'1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', 
         '2004', '2005', '2006', '2007', '2008', '2009', 
         '2010', '2011', '2012', '2013', '2014', '2015', '2016']


# List of states and D.C.
states = [
    'Alabama','Alaska','Arizona','Arkansas','California','Colorado',
    'Connecticut','Delaware','District Of Columbia','Florida','Georgia','Hawaii','Idaho', 
    'Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana',
    'Maine','Maryland','Massachusetts','Michigan','Minnesota',
    'Mississippi', 'Missouri','Montana','Nebraska','Nevada',
    'New Hampshire','New Jersey','New Mexico','New York',
    'North Carolina','North Dakota','Ohio',    
    'Oklahoma','Oregon','Pennsylvania','Rhode Island',
    'South Carolina','South Dakota','Tennessee','Texas','Utah',
    'Vermont','Virginia','Washington','West Virginia',
    'Wisconsin','Wyoming'
]

state_codes = {
    'Alabama' : '01',
    'Alaska' : '02',
    'Arizona' : '04',
    'Arkansas' : '05',
    'California' : '06',
    'Colorado' : '08',
    'Connecticut' : '09',
    'Delaware' : '10',
    'District Of Columbia' : '11',
    'Florida' : '12',
    'Georgia' : '13',
    'Hawaii' : '15',
    'Idaho' : '16', 
    'Illinois' : '17',
    'Indiana' : '18',
    'Iowa' : '19',
    'Kansas' : '20',
    'Kentucky' : '21',
    'Louisiana' : '22',
    'Maine' : '23',
    'Maryland' : '24',
    'Massachusetts' : '25',
    'Michigan' : '26',
    'Minnesota' : '27',
    'Mississippi' : '28',
    'Missouri' : '29',
    'Montana' : '30',
    'Nebraska' : '31',
    'Nevada' : '32',
    'New Hampshire' : '33',
    'New Jersey' : '34',
    'New Mexico' : '35',
    'New York' : '36',
    'North Carolina' : '37',
    'North Dakota' : '38',
    'Ohio' : '39',    
    'Oklahoma' : '40',
    'Oregon' : '41',
    'Pennsylvania' : '42',
    'Rhode Island' : '44',
    'South Carolina' : '45',
    'South Dakota' : '46',
    'Tennessee' : '47',
    'Texas' : '48',
    'Utah' : '49',
    'Vermont' : '50',
    'Virginia' : '51',
    'Washington' : '53',
    'West Virginia' : '54',
    'Wisconsin' : '55',
    'Wyoming' : '56'
    }

industries = ['information', 'finance', 'construction', 'manufacturing', 'transport_and_trade', 'all']

In [75]:
years

['2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016']

First, we'll be collecting 2008 data from the New York Times page on the election. In terms of formatting, we're hoping to get mainly the state and the number of votes for the Democratic and Republican party candidates (we'll be ignoring third-parties in this analysis).

In [76]:
'''
Converts data taken from the table in the html
    Input: a table row of voting data, in html
    Output: a list of data in a more desirable format
'''
def voter_html_to_data_2008(voter_html):
    data = [datum.get_text().strip() for datum in voter_html.find_all('td')]
    blue_votes = int(data[2][:-5].replace(',',''))
    red_votes = int(data[4][:-5].replace(',',''))
    return [blue_votes, red_votes]

def get_voter_data_2008():
    # Set up our data frame
    df = pd.DataFrame(columns=('election_year', 'state', 'dem_votes', 'rep_votes'))
    
    # Base url we'll be getting data from 
    base_url = 'http://elections.nytimes.com/2008/results/states/president/'
    
    # Get state names for url endings
    state_urls = sorted([state.lower().replace(' ', '-') for state in states])

    # Iterate through the states (except for Alaska and D.C.)
    num_states = 0
    for state in state_urls:
        if state != 'alaska' and state != 'district-of-columbia':
            # Get data from the site
            response = requests.get(base_url + state + '.html')
            election_soup = BeautifulSoup(response.text, 'html.parser')

            # Data for all states except for Alaska and D.C.
            data_rows = election_soup.find(id='winners-by-county-table').tbody.find_all('tr')
            
            # Data that every row will have (election year and state)
            header_data = ['2008', state]
            state_vote_lists = [voter_html_to_data_2008(row) for row in data_rows]
            state_vote_counts = [sum([vote[0] for vote in state_vote_lists]), sum([vote[1] for vote in state_vote_lists])]
            voter_data = header_data + state_vote_counts
            df.loc[num_states] = voter_data
            num_states += 1
    
    # Since Alaska and D.C. don't have counties, we process them slightly differently
    # First, Alaska
    alaska_html = requests.get('http://elections.nytimes.com/2008/results/states/alaska.html')
    alaska_election_soup = BeautifulSoup(alaska_html.text, 'html.parser')
    
    alaska_obama = alaska_election_soup.find(id='presidential-results-table').tbody.find_all('tr')[:2][1].find_all('td')
    alaska_mccain = alaska_election_soup.find(id='presidential-results-table').tbody.find_all('tr')[:2][0].find_all('td')
    alaska_blue_votes = int(alaska_obama[1].get_text().strip().replace(',',''))
    alaska_red_votes = int(alaska_mccain[2].get_text().strip().replace(',',''))
    alaska_data = ['2008', 'alaska', alaska_blue_votes, alaska_red_votes]
    df.loc[num_states] = alaska_data
    num_states += 1
    
    # Finally, D.C.
    dc_html = requests.get('http://elections.nytimes.com/2008/results/states/district-of-columbia.html')
    dc_election_soup = BeautifulSoup(dc_html.text, 'html.parser')

    dc_obama = dc_election_soup.find(id='presidential-results-table').tbody.find_all('tr')[:2][0].find_all('td')
    dc_mccain = dc_election_soup.find(id='presidential-results-table').tbody.find_all('tr')[:2][1].find_all('td')
    dc_blue_votes = int(dc_obama[2].get_text().strip().replace(',',''))
    dc_red_votes = int(dc_mccain[1].get_text().strip().replace(',',''))
    dc_data = ['2008', 'district-of-columbia', dc_blue_votes, dc_red_votes]
    df.loc[num_states] = dc_data
    num_states += 1
    
    return df

In [77]:
df_2008 = get_voter_data_2008()

In [78]:
df_2008

,election_year,state,dem_votes,rep_votes
0,2008,alabama,811764.0,1264879.0
1,2008,arizona,948648.0,1132560.0
2,2008,arkansas,418049.0,632672.0
3,2008,california,7441458.0,4554643.0
4,2008,colorado,1216793.0,1020135.0
5,2008,connecticut,994320.0,627688.0
6,2008,delaware,255394.0,152356.0
7,2008,florida,4143957.0,3939380.0
8,2008,georgia,1843452.0,2048244.0
9,2008,hawaii,324918.0,120309.0


For 2012, we'll switch to Politico, since the website formatting is slightly easier to parse than the interactive graphics on the New York Times site.

In [79]:
def get_voter_data_2012():
    # Set up our data frame
    df = pd.DataFrame(columns=('election_year', 'state', 'dem_votes', 'rep_votes'))
    
    # Base url we'll be getting data from
    base_url = 'http://www.politico.com/2012-election/results/president/'
    
    # Get state names for url endings
    state_urls = sorted([state.lower().replace(' ', '-') for state in states])
    
    # Once again, Alaska and D.C. are slightly different
    num_states = 0
    for state in state_urls:
        # Get data from site
        try:
            response = requests.get(base_url + state + '/')
        except ConnectionError:
            time.sleep(2)
            response = requests.get(base_url + state + '/')
        election_soup = BeautifulSoup(response.text, 'html.parser')
        header_data = ['2012', state]
        data = election_soup.find('div', class_='state-results-macro').table.tbody
        blue_votes = int(data.find(class_='party-democrat').find(class_='results-popular').get_text().strip().replace(',',''))
        red_votes = int(data.find(class_='party-republican').find(class_='results-popular').get_text().strip().replace(',',''))
        voter_data = header_data + [blue_votes, red_votes]
        df.loc[num_states] = voter_data
        num_states += 1
        
    return df

In [80]:
df_2012 = get_voter_data_2012()

In [81]:
df_2012

,election_year,state,dem_votes,rep_votes
0,2012,alabama,793620.0,1252453.0
1,2012,alaska,102138.0,136848.0
2,2012,arizona,930669.0,1143051.0
3,2012,arkansas,389699.0,638467.0
4,2012,california,6493924.0,4202127.0
5,2012,colorado,1238490.0,1125391.0
6,2012,connecticut,912531.0,631432.0
7,2012,delaware,242547.0,165476.0
8,2012,district-of-columbia,222332.0,17337.0
9,2012,florida,4235270.0,4162081.0


Finally, for 2016, we'll be using Politico again.

In [82]:
def voter_html_to_data_2016(voter_html):
    blue_votes = int(voter_html.find(class_='type-democrat').find(class_='results-popular').get_text().replace(',',''))
    red_votes = int(voter_html.find(class_='type-republican').find(class_='results-popular').get_text().replace(',',''))
    return [blue_votes, red_votes]

def get_voter_data_2016():
    # Set up our data frame
    df = pd.DataFrame(columns=('election_year', 'state', 'dem_votes', 'rep_votes'))
    
    # Base url we'll be getting data from
    base_url = 'http://www.politico.com/2016-election/results/map/president/'
    
    # Get state names for url endings
    state_urls = sorted([state.lower().replace(' ', '-') for state in states])
    
    # This time, Alaska and D.C. aren't different!
    num_states = 0
    for state in state_urls:
        # Get data from site
        response = requests.get(base_url + state + '/')
        election_soup = BeautifulSoup(response.text, 'html.parser')

        data = election_soup.select('section.content-group.election-intro')[0].find('div', class_='overall')
        header_data = ['2016', state]
        voter_data = header_data + voter_html_to_data_2016(data)
        df.loc[num_states] = voter_data
        num_states += 1
    
    return df

In [83]:
df_2016 = get_voter_data_2016()

In [84]:
df_2016

,election_year,state,dem_votes,rep_votes
0,2016,alabama,718084.0,1306925.0
1,2016,alaska,93007.0,130415.0
2,2016,arizona,936250.0,1021154.0
3,2016,arkansas,378729.0,677904.0
4,2016,california,7362490.0,3916209.0
5,2016,colorado,1208095.0,1136354.0
6,2016,connecticut,884432.0,668266.0
7,2016,delaware,235581.0,185103.0
8,2016,district-of-columbia,260223.0,11553.0
9,2016,florida,4485745.0,4605515.0


To aid in our analysis, we'll create a simple function to find the percent of the state that voted blue, given an election year.

In [85]:
def find_percent_blue(state, year):
    if year == 2008:
        df = df_2008
    elif year == 2012:
        df = df_2012
    else:
        df = df_2016
    df = df.set_index('state')
    dem_votes = df.at[state,'dem_votes']
    rep_votes = df.at[state, 'rep_votes']
    return dem_votes / (dem_votes + rep_votes)

In [86]:
print find_percent_blue('california', 2012)
print find_percent_blue('california', 2016)

0.60713285679
0.652778303597


Next, we'll begin gathering data from the BLS, specifically, data regarding employment by state in the industries most negatively impacted by the recession (construction, manufacturing, and transportation/trade), as well as overall employment, in thousands of employees. 

---

## Scraping Labor Statistics

In [87]:
import json, string
import requests, tqdm
from collections import defaultdict

To utilize the Bureau of Labor Statistics Public API is unintuitive, to say the least. Essentially, each unique dataset that the BLS keeps has a serial ID that correesponds to it. Querying the data from the API involves figuring out exactly which dataset you want, and then translating that to a serial ID. The code below allows us to input requests in a reasonable format like JSON and have that get converted to the serial ID that we need.

In [88]:
def opand(x,y): 
    return x and y


def nest(keys, d_keys, d):
    assert (len(keys) == len(d_keys))
    iter_format = lambda (i, d_key) : "for x%d in d['%s']" % (i, d_key)
    iters = string.join(map(iter_format, enumerate(d_keys)), sep=" ")
    arg_format = lambda (i) : '("' + keys[i] + '", x' + str(i) + ')'
    args = string.join(map(arg_format, range(len(keys))), sep=", ")
    return eval('[dict([' +  args +  ']) ' + iters + ']')


def request(series):
    """
    brief:
        - requests series in batches of 15, returns list of json_data, 
          one for each request
    args:
        - series : list of bls series id's
    """
    bls_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
    regKeys = ["bc2b9775e9794f37a23c0f6b2a4659b1", 
               "8874280f2cc142f3833754ad3cb6855c",
               "dacaff7ea67b466685622348e9807f8f"]
    json_data = []
    for i in tqdm.tqdm(range(0,len(series),50)):
        headers = {'Content-type': 'application/json'}
        data = json.dumps({"seriesid": series[i:i+50],
                           "startyear":"2004", 
                           "endyear":"2016", 
                           "registrationKey":regKeys[0]})
        try:
            response = requests.post(bls_url, data=data, headers=headers)
        except ConnectionError:
            time.sleep(2)
            response = requests.post(bls_url, data=data, headers=headers)
        json_data.append(json.loads(response.text))
    return json_data


def format_series(config):
    """
    brief:
        - formats a request for data from the bls state and area employment, 
          hours, and earnings database if config['prefix'] == 'SM'

        - formats a request for data from the bls local area unemployment 
          statistics database if config['prefix'] == 'SA'
    args:
        - config : {                   # http://www.bls.gov/help/hlpforma.htm#SM
                    'prefix' : 'SM',
                    'seasonal_adjustment' : 'S' or 'U',
                    'state_code' : '01' to '50', 
                    'area_code' : '00000' to '99999',
                    'supersector' : '00' to '99', 
                    'industry' : '000000' to '999999',     
                    'datatype' : '00' to '99'              
                    }
                or {                   # http://www.bls.gov/help/hlpforma.htm#LA
                    'prefix' : 'LA', 
                    'seasonal_adjustment' : 'S' or 'U',
                    'area_type' : 'ST' or 'MT', 
                    'state_code' : '01' to '50',
                    'area_code' : '000000000' or 'YYYYYYYYYYY'
                    'measure': '03' to '06'
                   }
    """
    keys = dict({'SM' : ['prefix','seasonal_adjustment','state_code','area_code','supersector','industry','datatype'],
                 'LA' : ['prefix', 'seasonal_adjustment','state_area_code', 'measure'],
                 })
    prefix = config['prefix']
    return string.join([config[key] for key in keys[prefix]], sep="")


def make_series(config):
    """
    brief:
        - provided a dictionary of bls series id parameters,
          construct and request each possible series_id given parameters
    args:
        - config : {
                    'prefix' : ['SM'],
                    'seasonal_adjustments' : ['S' or 'U'],
                    'state_codes' : [string],
                    'area_codes' : [{string -> string}],
                    'supersectors' : [string],
                    'industries' : [string],
                    'datatypes' : [string]
                    }
                or {
                    'prefix' : ['LA'],
                    'seasonal_adjustment' : ['S' or 'U'],
                    'area_types' : ['ST' or 'MT'],
                    'state_codes' : ['01' to '50']
                    'area_codes' : [{'ST', 'xx' --> 'STxx00000000000', 
                                    'MT', 'xx' --> 'MTxxYYYYYYYYYYY'}]
                    'measures': [string]
                   }
    """
    keys = dict({'SM' : ['prefix','seasonal_adjustment','state_code','area_code','supersector','industry','datatype'],
                 'LA' : ['prefix','seasonal_adjustment','state_area_code','measure'],
                 })
    param_keys = dict({'SM' : ['prefix','seasonal_adjustments','state_codes','area_codes','supersectors','industries','datatypes'],
                       'LA' : ['prefix','seasonal_adjustments','state_area_codes','measures'],
                       })
    prefix = config['prefix'][0]
    return [format_series(sid) for sid in nest(keys[prefix], param_keys[prefix], config)]

With that, now we can actually use the API to get some useful information. Here, with a provided listed of metro codes (BLS codes corresponding to different metropolitan areas in the U.S.), and using those, we can construct queries to fetch various employment statistics from different states and metropolitan areas.

In [89]:
config = {'prefix' : ['SM'],
          'seasonal_adjustments' : ['S', 'U'],
          'state_codes' : state_codes.values(),
          'area_codes' : ['00000'],
          'supersectors' : ['20', '30', '40', '50', '55', '00'],
          'industries' : ['000000'],
          'datatypes' : ['01'],
          }
           
json_data_list = request(make_series(config))

100%|██████████| 13/13 [01:13<00:00,  6.18s/it]


Now that we've retrieved our data from the BLS API, we'll format it into the Pandas dataframes we'll be performing analyses on in the next section.

In [90]:
def series_id_to_state(series_id):
    state_code_to_string = dict(zip(state_codes.values(), state_codes.keys()))
    return state_code_to_string[str(series_id[3:5])]

def series_id_to_industry(series_id):
    industry_code_to_string = dict({'50' : 'information',
                                    '55' : 'finance',
                                    '20' : 'construction',
                                    '30' : 'manufacturing',
                                    '40' : 'transport_and_trade',
                                    '00' : 'all',
                                   })
    return industry_code_to_string[series_id[10:12]]


industry_data = dict({'information' : dict(zip(states, [defaultdict(list) for i in range(len(states))])),
                      'finance' : dict(zip(states, [defaultdict(list) for i in range(len(states))])),
                      'construction' : dict(zip(states, [defaultdict(list) for i in range(len(states))])),
                      'manufacturing' : dict(zip(states, [defaultdict(list) for i in range(len(states))])),
                      'transport_and_trade' : dict(zip(states, [defaultdict(list) for i in range(len(states))])),
                      'all' : dict(zip(states, [defaultdict(list) for i in range(len(states))])),
                      })

# each json_data request has at between 1 <= # requests <= 50 in a list
for json_data in json_data_list:
    # each series in this list spans the years 2004:2016, and months 01:12
    for json_series in json_data['Results']['series']:
        series_id = json_series['seriesID']
        # the state, and industry is constant for each series
        state = series_id_to_state(series_id)
        industry = series_id_to_industry(series_id)
        # we want to average the values for each month for a given year
        for year in years:
            year_avg = []
            for entry in json_series['data']:
                if entry['year'] == year:
                    year_avg.append(float(entry['value']))
            year_avg = (0 if year_avg == [] else sum(year_avg) / len(year_avg))
            industry_data[industry][state][year].append(year_avg)

industry_dfs = dict()
for industry in industries:
    state_data = industry_data[industry]
    for state in states:
        year_data = state_data[state]
        coalesce = lambda year : sum(year_data[year]) / len(year_data[year]) if year_data[year] != [] else 0
        state_data[state] = [coalesce(year) for year in years]
    industry_dfs[industry] = pd.DataFrame(data=state_data, index=years, columns=states).transpose()

In [91]:
construction_df = industry_dfs['construction']
manufacturing_df = industry_dfs['manufacturing']
transport_and_trade_df = industry_dfs['transport_and_trade']
finance_df = industry_dfs['finance']
information_df = industry_dfs['information']
total_employment_df = industry_dfs['all']

In [92]:
construction_df

,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Alabama,102.816667,105.529167,109.845833,112.729167,108.508333,91.775000,87.204167,80.712500,79.608333,79.266667,79.879167,80.991667,79.727273
Alaska,17.691667,19.054167,18.695833,17.845833,17.425000,16.525000,16.550000,16.325000,17.070833,17.245833,17.800000,17.616667,16.931818
Arizona,191.708333,219.379167,240.379167,224.929167,186.066667,128.700000,111.558333,110.870833,116.050000,123.708333,125.129167,127.537500,135.836364
Arkansas,51.441667,54.645833,56.825000,56.287500,56.220833,51.562500,48.687500,47.287500,47.433333,45.466667,46.041667,49.008333,49.163636
California,850.229167,905.408333,933.595833,892.812500,788.037500,622.962500,559.920833,561.175000,589.854167,637.400000,674.266667,726.979167,763.404545
Colorado,151.241667,159.979167,167.783333,167.779167,161.820833,131.308333,115.066667,112.500000,115.912500,127.587500,142.258333,149.487500,161.968182
Connecticut,65.666667,65.954167,67.150000,68.629167,65.358333,54.537500,49.970833,51.433333,51.375000,53.420833,55.487500,57.420833,57.504545
Delaware,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
District Of Columbia,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Florida,572.741667,637.420833,682.179167,622.825000,515.895833,396.670833,350.695833,335.495833,342.066667,365.945833,397.300000,429.075000,454.204545


In [93]:
manufacturing_df

,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Alabama,145.775000,149.358333,151.437500,148.154167,142.141667,123.387500,118.150000,118.700000,121.812500,124.633333,126.591667,128.737500,130.650000
Alaska,12.270833,12.550000,13.170833,13.141667,13.020833,12.841667,12.725000,13.750000,14.033333,14.312500,14.483333,14.116667,14.304545
Arizona,176.954167,182.175000,185.620833,181.620833,173.008333,153.616667,148.325000,150.325000,154.866667,155.179167,156.612500,158.220833,159.004545
Arkansas,203.762500,201.945833,199.808333,190.608333,183.608333,163.700000,160.262500,158.941667,155.441667,152.833333,154.216667,154.883333,153.722727
California,1523.691667,1505.195833,1490.883333,1465.145833,1426.733333,1283.541667,1244.041667,1250.150000,1254.520833,1256.279167,1274.212500,1292.041667,1284.545455
Colorado,149.658333,148.345833,147.225000,145.191667,142.345833,128.016667,124.179167,128.104167,130.916667,132.766667,136.575000,141.337500,143.231818
Connecticut,196.912500,194.812500,193.041667,190.204167,186.612500,170.541667,164.858333,165.425000,163.958333,162.395833,159.391667,158.950000,159.672727
Delaware,34.566667,33.225000,33.562500,33.233333,31.725000,27.866667,25.933333,25.720833,25.662500,25.429167,26.129167,27.216667,27.345455
District Of Columbia,1.216667,1.050000,0.875000,0.845833,0.766667,0.641667,0.545833,0.525000,0.504167,0.500000,0.512500,0.545833,0.600000
Florida,410.641667,415.433333,416.454167,399.045833,371.091667,324.016667,309.075000,312.466667,317.404167,322.404167,331.908333,342.729167,353.804545


In [94]:
transport_and_trade_df

,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Alabama,376.895833,382.775000,387.908333,394.691667,389.520833,365.754167,361.437500,364.287500,365.979167,368.483333,372.850000,378.025000,381.772727
Alaska,62.062500,63.012500,63.708333,64.145833,64.616667,63.216667,62.758333,63.316667,63.829167,64.037500,64.766667,66.066667,66.831818
Arizona,462.670833,488.591667,512.220833,527.625000,516.800000,478.350000,467.858333,473.066667,476.370833,481.112500,492.791667,506.591667,515.681818
Arkansas,241.962500,245.500000,248.691667,250.041667,246.912500,235.529167,234.641667,238.154167,241.333333,241.212500,244.850000,252.208333,256.268182
California,2753.341667,2819.829167,2876.629167,2912.795833,2849.120833,2642.216667,2628.191667,2678.975000,2733.362500,2792.504167,2862.558333,2937.991667,2990.295455
Colorado,406.612500,412.954167,419.433333,429.154167,429.287500,403.862500,397.675000,401.770833,409.679167,420.233333,432.875000,445.550000,450.431818
Connecticut,304.033333,306.650000,307.145833,307.937500,305.708333,288.983333,285.854167,288.795833,291.266667,293.862500,295.708333,296.920833,297.154545
Delaware,80.804167,81.745833,82.554167,83.033333,81.145833,75.287500,74.487500,75.491667,75.837500,77.595833,79.650000,81.287500,81.668182
District Of Columbia,27.895833,27.779167,27.875000,27.695833,27.908333,26.850000,27.329167,27.450000,28.100000,29.062500,30.612500,32.229167,33.477273
Florida,1536.945833,1598.608333,1631.970833,1642.000000,1596.362500,1486.825000,1474.491667,1502.845833,1536.850000,1573.016667,1625.575000,1680.741667,1715.218182


In [95]:
finance_df

,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Alabama,97.062500,98.333333,98.779167,99.833333,99.116667,95.770833,92.054167,92.362500,92.629167,94.225000,94.741667,95.425000,98.631818
Alaska,12.241667,12.529167,12.583333,12.512500,12.216667,12.154167,12.220833,12.183333,12.187500,12.195833,12.095833,12.158333,12.409091
Arizona,166.304167,176.354167,185.120833,186.591667,178.366667,170.120833,167.575000,171.345833,176.650000,184.887500,189.808333,194.754167,204.640909
Arkansas,50.866667,51.425000,52.587500,53.137500,52.341667,50.433333,48.804167,48.541667,49.033333,49.550000,49.325000,49.283333,49.681818
California,894.779167,919.920833,927.312500,896.195833,841.554167,782.604167,759.670833,761.370833,772.829167,782.095833,781.670833,797.725000,809.663636
Colorado,154.554167,158.512500,160.441667,159.462500,155.616667,147.950000,144.325000,143.870833,146.670833,150.991667,153.908333,159.454167,161.786364
Connecticut,140.637500,142.291667,144.262500,144.620833,143.400000,137.633333,135.212500,134.991667,133.112500,130.541667,128.745833,130.029167,132.159091
Delaware,44.670833,45.220833,44.725000,45.266667,45.495833,43.912500,42.712500,42.220833,42.391667,43.862500,45.029167,46.633333,47.204545
District Of Columbia,30.583333,30.245833,29.458333,29.208333,28.158333,26.912500,26.891667,27.604167,28.208333,28.983333,30.125000,30.287500,30.609091
Florida,514.412500,539.875000,556.633333,551.158333,525.916667,487.383333,477.991667,486.512500,499.412500,512.704167,521.241667,534.920833,552.400000


In [96]:
information_df

,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Alabama,29.516667,29.212500,28.512500,28.025000,26.875000,25.120833,24.037500,23.125000,22.654167,22.608333,21.983333,21.466667,21.254545
Alaska,6.879167,6.954167,7.004167,6.925000,6.995833,6.566667,6.529167,6.329167,6.208333,6.158333,6.250000,6.275000,6.131818
Arizona,46.375000,44.775000,43.508333,41.575000,40.970833,38.133333,36.558333,37.275000,39.779167,41.962500,43.595833,45.312500,47.709091
Arkansas,19.800000,19.316667,18.612500,18.145833,17.400000,16.204167,15.320833,14.854167,14.479167,14.145833,13.591667,13.400000,13.790909
California,482.120833,473.712500,466.066667,471.166667,476.187500,441.166667,428.829167,430.591667,435.050000,449.087500,462.941667,482.770833,495.786364
Colorado,81.195833,76.904167,75.425000,76.354167,76.841667,74.654167,72.016667,71.358333,69.783333,69.850000,70.325000,70.729167,71.840909
Connecticut,38.991667,38.108333,37.916667,38.408333,37.783333,34.329167,31.720833,31.300000,31.295833,32.016667,32.058333,32.458333,33.586364
Delaware,3.529167,3.341667,3.350000,3.466667,3.516667,3.241667,2.975000,2.833333,2.725000,2.595833,2.458333,2.362500,2.436364
District Of Columbia,23.833333,22.541667,22.145833,21.916667,20.900000,19.058333,18.666667,18.337500,17.487500,17.029167,17.212500,17.012500,16.922727
Florida,164.304167,164.650000,163.154167,162.666667,157.191667,143.770833,137.079167,135.704167,133.575000,134.145833,136.145833,135.833333,133.659091


In [97]:
total_employment_df

,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Alabama,1901.733333,1944.858333,1979.662500,2005.725000,1992.212500,1886.420833,1870.708333,1870.154167,1885.183333,1902.675000,1922.541667,1947.037500,1965.472727
Alaska,303.179167,308.775000,313.904167,316.912500,321.170833,319.908333,324.045833,329.412500,334.795833,336.112500,337.666667,339.379167,338.818182
Arizona,2384.504167,2512.583333,2639.483333,2679.375000,2622.679167,2432.320833,2386.175000,2412.383333,2463.266667,2520.670833,2570.391667,2636.595833,2697.450000
Arkansas,1158.141667,1178.016667,1198.779167,1204.566667,1202.320833,1164.891667,1163.058333,1170.137500,1176.429167,1176.237500,1187.445833,1208.891667,1227.145455
California,14747.041667,15044.925000,15324.920833,15460.558333,15299.116667,14436.891667,14282.254167,14434.995833,14760.258333,15153.791667,15584.862500,16051.658333,16441.586364
Colorado,2179.520833,2225.937500,2279.400000,2331.216667,2350.470833,2245.533333,2222.262500,2258.779167,2313.116667,2382.083333,2464.812500,2541.466667,2605.272727
Connecticut,1649.862500,1662.291667,1680.920833,1698.395833,1699.062500,1626.479167,1607.954167,1624.141667,1637.620833,1650.037500,1661.295833,1673.658333,1684.422727
Delaware,425.079167,433.020833,438.220833,439.000000,436.783333,416.525000,413.779167,417.062500,419.454167,428.591667,438.533333,448.858333,459.277273
District Of Columbia,674.179167,682.237500,687.641667,693.733333,703.816667,701.500000,712.187500,726.220833,734.787500,748.308333,753.566667,766.433333,777.963636
Florida,7480.762500,7781.020833,7983.470833,7998.725000,7714.975000,7231.466667,7172.745833,7252.087500,7396.608333,7582.383333,7824.733333,8094.550000,8339.445455


Now that all of our data is properly formatted, we can perform our analysis. In the next section, we manipulate the dataframes we created with the goal of predicting whether a state will be red or blue in 2016. We do this by training an SVC on the voter and employment datasets for 2008 and 2012 that we generated.

## Analyzing Employment Data

First, we aggregate the employment of our blue collar industries into one big dataframe listing each state's blue collar employees in each election year.

In [98]:
def create_blue_collar_df(construction, manufacturing, transport_trade):
    df = pd.DataFrame({
            '2004': construction['2004'] + manufacturing['2004'] + transport_trade['2004'],
            '2008': construction['2008'] + manufacturing['2008'] + transport_trade['2008'],
            '2012': construction['2012'] + manufacturing['2012'] + transport_trade['2012'],
            '2016': construction['2016'] + manufacturing['2016'] + transport_trade['2016']
        })
    return df
    
blue_collar_df = create_blue_collar_df(construction_df, manufacturing_df, transport_and_trade_df)

In [102]:
blue_collar_df

,2004,2008,2012,2016
Alabama,625.487500,640.170833,567.400000,592.150000
Alaska,92.025000,95.062500,94.933333,98.068182
Arizona,831.333333,875.875000,747.287500,810.522727
Arkansas,497.166667,486.741667,444.208333,459.154545
California,5127.262500,5063.891667,4577.737500,5038.245455
Colorado,707.512500,733.454167,656.508333,755.631818
Connecticut,566.612500,557.679167,506.600000,514.331818
Delaware,115.370833,112.870833,101.500000,109.013636
District Of Columbia,29.112500,28.675000,28.604167,34.077273
Florida,2520.329167,2483.350000,2196.320833,2523.227273


We then use this dataframe and the total employment dataframe to make our SVM input dataframes.

In [111]:
def create_SVM_input(year, blue_collar_df, total_employment_df, df_4_years_ago):
    current_year = int(year)
    eight_years_ago = str(current_year - 8)
    df = pd.DataFrame({"blue_collar_change_last_8_years": (blue_collar_df[year] - blue_collar_df[eight_years_ago]) / blue_collar_df[eight_years_ago],
                       "total_employment_change_last_8_years": (total_employment_df[year] - total_employment_df[eight_years_ago]) / total_employment_df[eight_years_ago]})
    #x = list(df.index)
    df["proportion_blue_4_years_ago"] = df.index.map(lambda x: find_percent_blue(x.lower().replace(" ", "-"), current_year))
    return df

df_predict_2012 = create_SVM_input('2012', blue_collar_df, total_employment_df, df_2008)
df_predict_2016 = create_SVM_input('2016', blue_collar_df, total_employment_df, df_2012)

In [112]:
df_predict_2012

,blue_collar_change_last_8_years,total_employment_change_last_8_years,proportion_blue_4_years_ago
Alabama,-0.092868,-0.008703,0.387875
Alaska,0.031604,0.104284,0.427381
Arizona,-0.101098,0.033031,0.448792
Arkansas,-0.106520,0.015790,0.379023
California,-0.107177,0.000896,0.607133
Colorado,-0.072089,0.061296,0.523922
Connecticut,-0.105915,-0.007420,0.591032
Delaware,-0.120228,-0.013233,0.594444
District Of Columbia,-0.017461,0.089899,0.927663
Florida,-0.128558,-0.011249,0.504358


In [113]:
df_predict_2016

,blue_collar_change_last_8_years,total_employment_change_last_8_years,proportion_blue_4_years_ago
Alabama,-0.075013,-0.013422,0.354608
Alaska,0.031618,0.054947,0.416284
Arizona,-0.074614,0.028509,0.478312
Arkansas,-0.056677,0.020647,0.358430
California,-0.005065,0.074676,0.652778
Colorado,0.030237,0.108405,0.515300
Connecticut,-0.077728,-0.008616,0.569610
Delaware,-0.034174,0.051499,0.559995
District Of Columbia,0.188397,0.105350,0.957491
Florida,0.016058,0.080943,0.493413


Next we label each state with its result (1 for Democrat, 0 for Republican) in each of the last 3 presidential elections (2008, 2012, 2016).

In [131]:
results = {
    'Alabama': [0,0,0], 'Alaska': [0,0,0],
    'Arizona': [0,0,0], 'Arkansas': [0,0,0],
    'California': [1,1,1], 'Colorado': [1,1,1],
    'Connecticut': [1,1,1], 'Delaware': [1,1,1],
    'District Of Columbia': [1,1,1], 'Florida': [1,1,0],
    'Georgia': [0,0,0], 'Hawaii': [1,1,1],
    'Idaho': [0,0,0], 'Illinois': [1,1,1],
    'Indiana': [1,0,0], 'Iowa': [1,1,0],
    'Kansas': [0,0,0], 'Kentucky': [0,0,0],
    'Louisiana': [0,0,0], 'Maine': [1,1,1],
    'Maryland': [1,1,1],'Massachusetts': [1,1,1],
    'Michigan': [1,1,0],'Minnesota': [1,1,1],
    'Mississippi': [0,0,0], 'Missouri': [0,0,0],
    'Montana': [0,0,0], 'Nebraska': [0,0,0],
    'Nevada': [1,1,1], 'New Hampshire': [1,1,1],
    'New Jersey': [1,1,1],'New Mexico': [1,1,1],
    'New York': [1,1,1], 'North Carolina': [1,0,0],
    'North Dakota': [0,0,0],'Ohio': [1,1,0],    
    'Oklahoma': [0,0,0],'Oregon': [1,1,1],
    'Pennsylvania': [1,1,0],'Rhode Island': [1,1,1],
    'South Carolina': [0,0,0],'South Dakota': [0,0,0],
    'Tennessee': [0,0,0],'Texas': [0,0,0],
    'Utah': [0,0,0], 'Vermont': [1,1,1],
    'Virginia': [1,1,1],'Washington': [1,1,1],
    'West Virginia': [0,0,0], 'Wisconsin': [1,1,0],
    'Wyoming': [0,0,0]
}

def provide_labels(state, year):
    #ret = []
    #for k,v in results.iteritems():
    #    res = v[(year - 2008) / 4]
    #    print (k + " was " + str(res) + " " + str(year))
    #    ret.append(res)
#    return np.array([v[(year - 2008) / 4] for k, v in results.iteritems()])
    return results[state][(year - 2008) / 4]
    
# df_predict_2012["alphabetical_state_results_2008"] = provide_labels(2008)
#print provide_labels(2012)
df_predict_2012["alphabetical_state_results_2012"] = df_predict_2012.index.map(lambda state: provide_labels(state, 2012))
df_predict_2016["alphabetical_state_results_2016"] = df_predict_2016.index.map(lambda state: provide_labels(state, 2016))

In [133]:
df_predict_2012

,blue_collar_change_last_8_years,total_employment_change_last_8_years,proportion_blue_4_years_ago,alphabetical_state_results_2012
Alabama,-0.092868,-0.008703,0.387875,0
Alaska,0.031604,0.104284,0.427381,0
Arizona,-0.101098,0.033031,0.448792,0
Arkansas,-0.106520,0.015790,0.379023,0
California,-0.107177,0.000896,0.607133,1
Colorado,-0.072089,0.061296,0.523922,1
Connecticut,-0.105915,-0.007420,0.591032,1
Delaware,-0.120228,-0.013233,0.594444,1
District Of Columbia,-0.017461,0.089899,0.927663,1
Florida,-0.128558,-0.011249,0.504358,1


In [134]:
df_predict_2016

,blue_collar_change_last_8_years,total_employment_change_last_8_years,proportion_blue_4_years_ago,alphabetical_state_results_2016
Alabama,-0.075013,-0.013422,0.354608,0
Alaska,0.031618,0.054947,0.416284,0
Arizona,-0.074614,0.028509,0.478312,0
Arkansas,-0.056677,0.020647,0.358430,0
California,-0.005065,0.074676,0.652778,1
Colorado,0.030237,0.108405,0.515300,1
Connecticut,-0.077728,-0.008616,0.569610,1
Delaware,-0.034174,0.051499,0.559995,1
District Of Columbia,0.188397,0.105350,0.957491,1
Florida,0.016058,0.080943,0.493413,0


Now that we've extracted all the necessary training data, we train an SVM on the features and labels of our training set.

In [143]:
# classifier for training examples, which are 2012 states and labels
def learn_classifier(training_features, training_labels):
    svc = SVC()
    svc.fit(training_features, training_labels)
    return svc

trained_svc_industry_to_total = learn_classifier(df_predict_2012[["blue_collar_change_last_8_years", 
                                               "total_employment_change_last_8_years"]], 
                               df_predict_2012["alphabetical_state_results_2012"])

trained_svc_industry_to_vote = learn_classifier(df_predict_2012[["blue_collar_change_last_8_years",
                                               "proportion_blue_4_years_ago"]], 
                               df_predict_2012["alphabetical_state_results_2012"])

trained_svc_total_to_vote = learn_classifier(df_predict_2012[["total_employment_change_last_8_years", 
                                               "proportion_blue_4_years_ago"]], 
                               df_predict_2012["alphabetical_state_results_2012"])

In [144]:
trained_svc_industry_to_total

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [145]:
trained_svc_industry_to_vote

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [146]:
trained_svc_total_to_vote

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Finally, we run our classifier on our testing set, the states in the 2016 election.

In [148]:
def eval_classifier(svc, testing_features, testing_labels):
    predicted_labels = svc.predict(testing_features)
    print predicted_labels
    return float(np.sum(predicted_labels == testing_labels)) / testing_features.shape[0]

accuracy_industry_to_total = eval_classifier(trained_svc_industry_to_total, 
                                             df_predict_2016[["blue_collar_change_last_8_years", 
                                                        "total_employment_change_last_8_years"]], 
                           df_predict_2016["alphabetical_state_results_2016"])

accuracy_industry_to_vote = eval_classifier(trained_svc_industry_to_vote, 
                                            df_predict_2016[["blue_collar_change_last_8_years",
                                                        "proportion_blue_4_years_ago"]], 
                           df_predict_2016["alphabetical_state_results_2016"])

accuracy_total_to_vote = eval_classifier(trained_svc_total_to_vote, 
                                         df_predict_2016[["total_employment_change_last_8_years", 
                                                        "proportion_blue_4_years_ago"]], 
                           df_predict_2016["alphabetical_state_results_2016"])


print accuracy_industry_to_total, accuracy_industry_to_vote, accuracy_total_to_vote

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 1 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 0 1 0
 1 1 1 0 0 0 0 0 1 1 1 0 1 0]
[0 0 1 0 1 1 1 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 0 1 0
 1 1 1 0 0 0 1 0 1 1 1 0 1 0]
0.411764705882 0.843137254902 0.803921568627
